In [30]:
import pandas as pd
import os
import glob

In [31]:
data_path = '/Users/sonynka/HTW/MasterArbeit/data/fashionid//'

In [32]:
df = pd.read_csv(os.path.join(data_path, 'data.csv'), sep=';')

In [33]:
df.head()

,img_path,attributes,brand,category,color,id,img_url,model_img_urls,name,product_url
0,blusen/203796.jpg,"Damen Hemdbluse von Polo Ralph Lauren, Baumwol...",Polo Ralph Lauren,blusen,white,203796,https://img.fidcdn.net/r17/product/polo-ralph-...,https://img.fidcdn.net/r17/product/polo-ralph-...,Hemdbluse mit Logo-Stickerei,https://www.fashionid.de/polo-ralph-lauren/dam...
1,blusen/203931.jpg,"Damen Hemdbluse von Polo Ralph Lauren, Reine B...",Polo Ralph Lauren,blusen,blue,203931,https://img.fidcdn.net/r17/product/polo-ralph-...,https://img.fidcdn.net/r17/product/polo-ralph-...,Custom Fit Hemdbluse mit Logo-Stickerei,https://www.fashionid.de/polo-ralph-lauren/dam...
2,blusen/203932.jpg,"Damen Hemdbluse von Polo Ralph Lauren, Reine B...",Polo Ralph Lauren,blusen,pink,203932,https://img.fidcdn.net/r17/product/polo-ralph-...,https://img.fidcdn.net/r17/product/polo-ralph-...,Hemdbluse mit Button-Down-Kragen,https://www.fashionid.de/polo-ralph-lauren/dam...
3,blusen/203933.jpg,"Damen Hemdbluse von Polo Ralph Lauren, Reine B...",Polo Ralph Lauren,blusen,white,203933,https://img.fidcdn.net/r17/product/polo-ralph-...,https://img.fidcdn.net/r17/product/polo-ralph-...,Custom Fit Hemdbluse mit Logo-Stickerei,https://www.fashionid.de/polo-ralph-lauren/dam...
4,blusen/203935.jpg,"Damen Hemdbluse von Polo Ralph Lauren, Reine B...",Polo Ralph Lauren,blusen,blue,203935,https://img.fidcdn.net/r17/product/polo-ralph-...,https://img.fidcdn.net/r17/product/polo-ralph-...,Hemdbluse mit Streifenmuster,https://www.fashionid.de/polo-ralph-lauren/dam...


In [34]:
print('Data points', df.shape[0])
print(df.groupby('category')['img_path'].count())

Data points 11857
category
blusen             1624
hosen              1059
jacken              710
jeans               963
kleider            2899
pullover-strick    1914
shirts             2688
Name: img_path, dtype: int64


# Clean data
## Remove Duplicates

Drop duplicated rows which could occured during scraping data and appending to csvs.

If the count of img_path is more than unique, drop the duplicated img_path. This can occurr, if the same image is assigned to two different colors. In that case, just keep the first color.

*Note: We apply this to img_path, since it is acceptable that img_url is duplicated -> one image can be assigned to two different subcategories, such as Kurze Kleider und Cocktailkleider. If one image should only have one subcategory assigned, duplicated should be dropped on img_url not on img_path.*

In [35]:
print('Shape', df.shape)
print('Without duplicates', df.drop_duplicates().shape)

Shape (11857, 10)
Without duplicates (11857, 10)


In [36]:
df[df.img_path.isin(df[df['img_path'].duplicated()]['img_path'])].sort_values('img_path')

,img_path,attributes,brand,category,color,id,img_url,model_img_urls,name,product_url


In [37]:
df = df.groupby('img_path').first().reset_index()

## Check images and CSVs consistency
### CSV -> image path check
Make sure all the image paths in the csvs actually exist in the folder structure.

In [38]:
df['exists'] = [True if os.path.exists(os.path.join(data_path, str(x))) else False for x in df['img_path']]
not_exist_imgs_count = df[df['exists'] == False].shape[0]
print('Not existing images', not_exist_imgs_count)
print(df[df['exists'] == False])

Not existing images 0
Empty DataFrame
Columns: [img_path, attributes, brand, category, color, id, img_url, model_img_urls, name, product_url, exists]
Index: []


In [39]:
df = df[df['exists'] == True]

In [40]:
df = df.drop('exists', axis=1)

### image path -> CSV check
Make sure all the images in the folders have an entry in the csv.

In [41]:
for category in os.listdir(data_path):
    
    category_path = os.path.join(data_path, category)
    if os.path.isdir(category_path):
        print(category)
        img_names = [os.path.join(category, o) for o in os.listdir(category_path)]
        row_names = df[df['category'] == category]['img_path']
        diff_imgs = list(set(img_names).difference(row_names))
        print('Diff', len(diff_imgs))
        
#         for img in diff_imgs:
#             os.remove(os.path.join(data_path, img))
            
        print('Diff', len(diff_imgs))
    

jacken
Diff 0
Diff 0
blusen
Diff 0
Diff 0
pullover-strick
Diff 0
Diff 0
shirts
Diff 0
Diff 0
jeans
Diff 0
Diff 0
roecke
Diff 0
Diff 0
kleider
Diff 0
Diff 0
hosen
Diff 0
Diff 0


In [29]:
df.to_csv(os.path.join(data_path, 'data.csv'), sep=';', encoding='utf-8', index=False)

In [ ]:
df.head()